<a href="https://colab.research.google.com/github/MasslessAI/narratelab/blob/master/exp_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

In [1]:
from google.colab import drive

drive.mount('gdrive', force_remount=True)

root_dir = '/content/gdrive/'

gdrive_path = root_dir + 'MyDrive/narratelab/exp_1'

!pip install spacy==3.1.0  keybert[spacy] sentence-transformers redditcleaner psaw pandas loguru bertopic[spacy] distinctipy colour tqdm
!python -m spacy download en_core_web_trf
!python -m spacy download en_core_web_sm

Mounted at gdrive
     |████████████████████████████████| 6.4MB 4.2MB/s 
     |████████████████████████████████| 92kB 11.8MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 624kB 36.4MB/s 
     |████████████████████████████████| 10.1MB 43.4MB/s 
     |████████████████████████████████| 460kB 32.1MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 215kB 46.5MB/s 
     |████████████████████████████████| 2.5MB 43.1MB/s 
     |████████████████████████████████| 1.2MB 44.9MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 13.2MB 23.6MB/s 
     |████████████████████████████████| 6.4MB 40.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 51kB 8.5MB/s 
     |███████████████████

# Scrap Reddit Data

Using pushshift api to quickly search for submissions where title contains **question-indicative phrases/words**. Only retrieve submissions whose **num_comments > 1**, this filters out most of the ads.

refer to [Content Ideas From Reddit](https://timothywangdev.github.io/knowledge/Content%20Marketing/content-ideas-from-reddit)

In [3]:
import datetime as dt
from datetime import date, datetime
from loguru import logger
import pandas as pd
from psaw import PushshiftAPI
import time
import os
import redditcleaner
import re
import base64
import IPython
from distinctipy import distinctipy
import random
from colour import Color

api = PushshiftAPI()

QUESTION_WORDS = [
  "what",
  "when",
  "where",
  "who",
  "whom",
  "which",
  "whose",
  "why",
  "how",
  "wonder",
  "want",
  "is anyone",
  "does anyone",
  "any tips",
  "advice",
  "suggestion",
  "suggestions",
  "suggest",
  "ideas on",
  "need help",
  "needs help",
  "need your help",
  "serious help",
  "please help",
  "challenge",
  "challenges",
  "can't stand",
  "struggle",
  "struggling",
  "can't figure out",
  "help me",
  "hardest part",
  "would appreciate",
  "would really appreciate",
  "any guidance",
  "no idea",
  "confused with",
  "new to",
  "is there any way"
]

# (Optional) Do some experiments on question words and psaw filters

In [ ]:

# do some exps on question words and psaw filters
_QUESTION_WORDS=["ideas on", "struggle with", "suggestion"]
title_query = '|'.join(map(lambda x: '"{}"'.format(x), _QUESTION_WORDS))
print(title_query)

gen = list(
        api.search_submissions(
            title=title_query,
            is_self=True,
            is_original_content=True, 
            subreddit='content_marketing',
            num_comments=">1",
            filter=['title', 'selftext', 'author', 'permalink', 'num_comments', 'score', 'total_awards_received',
                    'upvote_ratio'],
            sort='asc', sort_type='created_utc', limit=500))

titles = [item.d_['title'] for item in gen]

# generate distinct hsl colors
bg_color_list = distinctipy.get_colors(50)
inverted_color_list = distinctipy.invert_colors(bg_color_list)
text_color_list = [distinctipy.get_text_color(color) for color in bg_color_list]

# convert colors to hex
bg_color_list = [Color(rgb=color).hex for color in bg_color_list]
inverted_color_list = [Color(rgb=color).hex for color in inverted_color_list]
text_color_list = [Color(rgb=color).hex for color in text_color_list]

def colored(word, color_idx=0):
  color = bg_color_list[color_idx]
  return '<span style="display: inline-block; font-weight: 600; font-size: 16px; border-radius: 15%; padding-left: 5px; padding-right: 5px; margin: 4px 2px; background-color: {}; color:{};">{}</span>'.format(bg_color_list[color_idx], text_color_list[color_idx], word)
  
colored_titles = []
for title in titles:
  _title = title.lower()
  for idx, question_word in enumerate(_QUESTION_WORDS):
    _title = _title.replace(question_word, colored(question_word, color_idx = idx))
  colored_titles.append('<li>' + _title + "</li>")

IPython.display.HTML("<ol>" +' '.join(colored_titles) + "</ol>")

"ideas on"|"struggle with"|"suggestion"


Only retrieve submissions that are text submissions, and filter out deleted submissions and submissions whose author is banned.

We keep the following fields:

* title
* selftext
* author
* permalink
* num_comments <font color='blue'>*</font>
* score <font color='blue'>*</font>
* upvote_ratio <font color='blue'>*</font>
* total_awards_received <font color='blue'>*</font>

<font color='blue'>*</font> *Used for future ranking purpose*

<mark>TODO</mark> 
 - [ ] Use praw to get latest metadata (pushshift only updates data in for the first 24hr)


In [ ]:
start_epoch = int(dt.datetime(2018, 1, 1).timestamp())
#end_epoch = int(dt.datetime(2020, 3, 1).timestamp())
end_epoch = int(time.time())
total = 0

SUBREDDIT = 'ethereum'
DATA_FILE_NAME = gdrive_path + '/reddit_submission_{}_{}_{}.tsv'.format(
    SUBREDDIT, datetime.fromtimestamp(start_epoch).strftime("%Y_%m_%d"),
    datetime.fromtimestamp(end_epoch).strftime("%Y_%m_%d"))

title_query = '|'.join(map(lambda x: '"{}"'.format(x), QUESTION_WORDS))
while True:
    gen = list(
        api.search_submissions(
            after=start_epoch, 
            before=end_epoch,
            title=title_query,
            is_self=True,
            is_original_content=True,
            subreddit=SUBREDDIT,
            # note the num_comments is only updated for the first 24hr
            # may need to use praw to get latest meta data
            num_comments=">1",
            filter=['title', 'selftext', 'author', 'permalink', 'num_comments', 'score', 'total_awards_received',
                    'upvote_ratio'],
            sort='asc', 
            sort_type='created_utc', 
            limit=500))

    if len(gen) == 0:
        break

    def submission_filter(submission):
        if 'title' not in submission:
            return False
        if 'selftext' not in submission:
            return False
        if len(submission['selftext']) == 0:
            # if submission is deleted, the psaw returns empty str
            return False
        if 'author' not in submission:
            return False
        if submission['author'] == "[deleted]":
            return False
        if any(submission['selftext'] == x for x in ["[removed]", "[deleted]"]):
            return False
        return True

    def prepare_data(data):
        # some of the fields may be missing
        # must manually set an init value to avoid
        # generating invalid csv
        _data = {
            'title': '',
            'selftext': '',
            'author': '',
            'permalink': '',
            'num_comments': 0,
            'score': 0,
            'total_awards_received': 0,
            'upvote_ratio': 1.0,
            'created_utc': None
        }

        for key in _data:
            if key in data and data[key] is not None:
                _data[key] = data[key]

        return _data


    items = map(prepare_data, [item.d_ for item in gen])

    items = list(filter(submission_filter, items))
    df = pd.DataFrame(items)

    # clean data
    def clean(text):
        text = text.lower()
        # remove reddit styles
        text = redditcleaner.clean(
            text, quote=False, bullet_point=False, link=False, strikethrough=False, spoiler=False, code=False,
            superscript=False, table=False)

        # refer to https://towardsdatascience.com/cleaning-text-data-with-python-b69b47b97b76
        # Remove unicode characters
        text = text.encode('ascii', 'ignore').decode()

        # Remove Hashtags
        text = re.sub("#\S+", " ", text)

        # Remove markdown links
        text = re.sub(r"\[(.+)\]\(.+\)", r"\1", text)

        # Remove other urls
        text = re.sub(r"http\S+", " ", text)

        # remove text inside brackets
        text = re.sub("\(.*?\)"," ", text)
        text = re.sub("\[.*?\]"," ", text)

        # remove quotes
        # remove brackets
        # remove semicolon
        text = re.sub(r'[\t()[\]\"*:\\]',' ', text)

        # remove non-ascii chars
        text = re.sub(r"[^\x00-\x7F]+",' ', text)

         # Replace the over spaces# if submission is deleted, the psaw returns NaN, must check if 
            # it's a valid string
        text = re.sub('\s{2,}', " ", text)

        return text

    if len(df) > 0:
      df['title'] = df['title'].map(clean)
      df['selftext'] = df['selftext'].map(clean)

      if not os.path.isfile(DATA_FILE_NAME):
          df.to_csv(DATA_FILE_NAME, sep='\t', header='column_names', index=False, quoting=3)
      else:  # else it exists so append without writing the header
          df.to_csv(DATA_FILE_NAME, sep='\t', mode='a', header=False, index=False, quoting=3)

      start_epoch = items[-1]['created_utc']
      total += len(items)

      logger.info('Added {} Total {} Last created_utc {}'.format(
        len(items), total, date.fromtimestamp(start_epoch)))

      time.sleep(1)
    else:
      break

2021-07-15 05:38:12.941 | INFO     | __main__:<module>:129 - Added 403 Total 403 Last created_utc 2018-03-13
2021-07-15 05:38:24.498 | INFO     | __main__:<module>:129 - Added 404 Total 807 Last created_utc 2018-09-05
2021-07-15 05:38:32.839 | INFO     | __main__:<module>:129 - Added 364 Total 1171 Last created_utc 2019-06-02
2021-07-15 05:38:42.502 | INFO     | __main__:<module>:129 - Added 288 Total 1459 Last created_utc 2020-06-16
2021-07-15 05:38:52.743 | INFO     | __main__:<module>:129 - Added 279 Total 1738 Last created_utc 2021-01-11
2021-07-15 05:38:58.958 | INFO     | __main__:<module>:129 - Added 337 Total 2075 Last created_utc 2021-04-04
2021-07-15 05:39:07.158 | INFO     | __main__:<module>:129 - Added 348 Total 2423 Last created_utc 2021-05-21
2021-07-15 05:39:11.333 | INFO     | __main__:<module>:129 - Added 181 Total 2604 Last created_utc 2021-07-05


# Load Scrapped Reddit Data

Load submissions, filter out ads, and classified them by wh-words

From observation, some **submissions are ads**, which often contain the following symbols/phrases

```
'-', ':', ';' 
your business(es)
help you
case study
ALL CAPITAL chars
how i
dollar symbol $ (e.g turn $100 into 100k)
step-by-step
here is how
here's how
part 1/2/3
top [number] things
[number] reasons
ultimate guide
cheat sheet
cheatsheet
infographic
by [year]
ama
```

<mark>TODO</mark> 
 - [x] Add more ad-indicative phrases
 - [x] Use num_comments to filter ads

In [4]:
df = pd.read_csv(gdrive_path +"/reddit_submission_cars_2016_01_01_2021_07_15.tsv", sep='\t', quoting=3)

# data cleanup
print('before clean-up # rows: {}'.format(len(df)))
cleaned_rows = []

AD_INDICATIVE_PHRASES = [
  "your business", 
  "your businesses", 
  "help you", 
  "case study"
  "how i",
  "$",
  "step-by-step",
  "here is how",
  "here's how",
  "part 1",
  "part 2",
  "part 3",
  "ultimate guide",
  "cheatsheet",
  "infographic",
  "ama"
]

for index, row in df.iterrows():
    row_title = row['title'].lower()
    if not any(x in ['-', ':', ';'] for x in row_title) and not any(phrase in row_title for phrase in AD_INDICATIVE_PHRASES) and "?" in row_title:
        '''
        1. must contain '?'
        2. can only contain alphanumeric, punctuations and space
        3. should not contain '-', ':', ';' which indicates ads
        '''
        cleaned_rows.append(index)

df = df[df.index.isin(cleaned_rows)].reset_index()
print('after clean-up # rows: {}'.format(len(df)))

print(df.head(5).to_markdown())

before clean-up # rows: 6295
after clean-up # rows: 4295
|    |   index | title                                                                                  | selftext                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

# Classified submissions by wh-words in title, extract keywords and aggregate stats

In [5]:
import pprint
from keybert import KeyBERT
import spacy
from tqdm import tqdm 

spacy.require_gpu()
nlp = spacy.load("en_core_web_trf")
kw_model = KeyBERT(model='paraphrase-MiniLM-L6-v2')

title_cat = []
keywords = []
lemmatized_combined_text = []
df['combined_text'] = df['title'] +' '+ df['selftext']
docs = list(nlp.pipe(df['combined_text']))
for index, row in tqdm(df.iterrows(), position=0, leave=True, total=len(df)):
    title_cat.append('NO_WH_WORD')  
    for wh_word in QUESTION_WORDS:
        if wh_word in row['title'].lower():
            title_cat[-1] = wh_word
            break
    doc = docs[index]

    doc_lemmas = ' '.join([token.lemma_ for token in doc])
    lemmatized_combined_text.append(doc_lemmas)

    entities = [ent.text for ent in doc.ents]

    # extract keywords from lemmatized text
    doc_keywords = kw_model.extract_keywords(doc_lemmas)
    doc_keywords = [keyword[0] for keyword in doc_keywords]

    # combine both extracted keywords and entities
    # and remove duplicates
    all_keywords = list(set(doc_keywords + entities))

    keywords.append(all_keywords)

df['title_cat'] = title_cat
df['keywords'] = keywords
df['lemmatized_combined_text'] = lemmatized_combined_text

cat_stats = {
    'cat': QUESTION_WORDS,
    'num_docs': [],
    'total_score': [],
    'total_comments': []
}

# print out number of docs per category
for wh_word in QUESTION_WORDS:
    _df = df[df['title_cat'] == wh_word]
    cat_stats['num_docs'].append(len(_df))
    cat_stats['total_score'].append(_df['score'].sum())
    cat_stats['total_comments'].append(_df['num_comments'].sum())
  
cat_stats_df = pd.DataFrame(data=cat_stats)

print('\n', cat_stats_df.to_markdown())
  

100%|██████████| 4295/4295 [01:47<00:00, 40.14it/s]



 |    | cat                     |   num_docs |   total_score |   total_comments |
|---:|:------------------------|-----------:|--------------:|-----------------:|
|  0 | what                    |       2114 |         38957 |           122404 |
|  1 | when                    |        186 |          1614 |             4044 |
|  2 | where                   |        176 |          2225 |             3310 |
|  3 | who                     |        107 |          2828 |             5449 |
|  4 | whom                    |          0 |             0 |                0 |
|  5 | which                   |        134 |          1992 |             6133 |
|  6 | whose                   |          0 |             0 |                0 |
|  7 | why                     |        387 |          5707 |            13385 |
|  8 | how                     |        861 |          8632 |            19288 |
|  9 | wonder                  |          2 |            10 |              117 |
| 10 | want               

# Pick a wh-word category

In [6]:
df_selection = df[df['title_cat'] == 'what'].copy().reset_index()
print(df_selection.head(10).to_markdown())

|    |   level_0 |   index | title                                                                                         | selftext                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

# Keywords Analysis

In [14]:
from gensim.corpora import Dictionary
from collections import defaultdict

# Create a id2word dictionary
_dict = Dictionary(df_selection['keywords'])
print('dict size (unfiltered) {}'.format(len(_dict)))


# Filtering Extremes
_dict.filter_extremes(no_below=2, no_above=.99)
_dict.compactify()
print('dict size {}'.format(len(_dict)))

keywords_stats = defaultdict(lambda: {
    'token': '',
    'num_comments': 0,
    'score': 0,
    'doc_freq': 0,
    'docs': []
})

token2doc = defaultdict(list)
for idx, row in df_selection.iterrows():
    for token in row['keywords']:
        token2doc[token].append(idx)
        keywords_stats[token]['num_comments'] += row['num_comments']
        keywords_stats[token]['score'] += row['score']

for token, id in _dict.token2id.items():
    keywords_stats[token]['token']=token
    keywords_stats[token]['doc_freq'] = _dict.dfs[id]
    keywords_stats[token]['docs'] = token2doc[token]

keywords_stats_df = pd.DataFrame(list(keywords_stats.values()))

# sort by num_comments
keywords_stats_df =keywords_stats_df.sort_values(by=['num_comments'], ascending=False)

print(keywords_stats_df.head(50).to_markdown())

dict size (unfiltered) 5423
dict size 1672
|      | token              |   num_comments |   score |   doc_freq | docs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

# Topic Modeling with BERTopic

In [15]:
from bertopic import BERTopic

topic_model = BERTopic(n_gram_range=(1, 1), nr_topics="auto", calculate_probabilities=True, min_topic_size=5)
_docs = [' '.join(list(map(lambda x: '_'.join(x.split()), keywords))) for keywords in list(df_selection['keywords'])]

topics, probabilities = topic_model.fit_transform(_docs)
#topics, probabilities = topic_model.fit_transform(df_selewhyction['lemmatized_combined_text'])
topic_info_df = topic_model.get_topic_info()
print(topic_info_df.to_markdown())

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


|    |   Topic |   Count | Name                                                                    |
|---:|--------:|--------:|:------------------------------------------------------------------------|
|  0 |      -1 |     913 | -1_vehicle_toyota_engine_truck                                          |
|  1 |       0 |     148 | 0_insurance_sell_dealership_license                                     |
|  2 |       1 |     117 | 1_bmw_honda_volvo_mercedes                                              |
|  3 |       2 |      79 | 2_tire_brake_wheel_rim                                                  |
|  4 |       3 |      63 | 3_regret_kind_shit_sexy                                                 |
|  5 |       4 |      54 | 4_sound_noise_loud_tune                                                 |
|  6 |       5 |      50 | 5_mustang_gt_ford_mustanggt                                             |
|  7 |       6 |      49 | 6_advice_buy_recommend_mind                                     

In [16]:
from collections import defaultdict

docs_by_topics = defaultdict(list)

for index, topic in enumerate(topics):
  docs_by_topics[topic].append({
      'docIdx': index,
      'prob': probabilities[index][topic],
      'num_comments':df_selection['num_comments'][index],
      'score': df_selection['score'][index],
      'title': df_selection['title'][index],
      'keywords': df_selection['keywords'][index]
  })

# sort doc by their topic prob
for index, topic in enumerate(topics):
  docs_by_topics[topic].sort(key=lambda x: x['prob'], reverse=True)


In [17]:

for idx, row in topic_info_df.iterrows():
  print(row.to_frame().T)
  topic_docs_df = pd.DataFrame(docs_by_topics[row['Topic']])
  print(topic_docs_df.head(50).to_markdown())



  Topic Count                            Name
0    -1   913  -1_vehicle_toyota_engine_truck
|    |   docIdx |   prob |   num_comments |   score | title                                                                                                                                                                  | keywords                                                                                                                                                                                                              |
|---:|---------:|-------:|---------------:|--------:|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
|  0 |        0 |      0 |    

In [ ]:
topics_over_time = topic_model.topics_over_time(df_selection['title'], topics, df_selection['created_utc'], nr_bins=20)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_heatmap()

In [ ]:
topic_model.visualize_hierarchy()